In [ ]:
import os
import numpy as np
import pandas as pd
from typing import Any

from exp_spec_info import *
from plot_info import *
from select_data import *

In [ ]:
# Data pickle path
extracted_data_path = "C:\\Users\\dosre\\dev\\thesis-data\\extracted_data.pkl"

# Processed data paths
extended_data_path = "C:\\Users\\dosre\\dev\\thesis-data\\extended_data.pkl"
median_data_path = "C:\\Users\\dosre\\dev\\thesis-data\\median_data.pkl"

##### Extend Data with Processed Elements

In [ ]:
data = pd.read_pickle(extracted_data_path)

# Extend with final relative residual
rel_res_ext = []
for setup in SETUPS:
    for matrix in SETUP_MATRIX_MAPPING[setup]:
        for restart_param in RESTART_PARAMS:
            for exp_iter in range(N_EXPERIMENT_ITERATIONS):

                sub_data = df_sel_setup_matrix_restart(
                    data, setup, matrix, restart_param
                )
                sub_data = sub_data[sub_data["experiment_iter"] == exp_iter]

                rel_res = sub_data["inner_relres"].apply(
                    lambda relres_arr: relres_arr[-1]
                )
                if len(rel_res) > 0:
                    rel_res_ext.append(rel_res)

rel_res_ext = pd.concat(rel_res_ext)
rel_res_ext.name = "final_relres"

extended_data = data.join((rel_res_ext))


def rel_res_frac_err(target_relres: Any, baseline_relres: Any) -> Any:
    return (target_relres-baseline_relres)/baseline_relres


def mean_rel_res_frac_err(
    target_relres: np.array, baseline_relres: np.array
) -> float:
    low_n = min(len(target_relres), len(baseline_relres))
    return np.mean(
        rel_res_frac_err(target_relres[:low_n], baseline_relres[:low_n])
    )


# Extend with values based on FP32 and FP64
rel_time_fp64_ext = []
rel_res_frac_err_fp64_ext = []
outer_mean_rel_res_frac_err_fp64_ext = []
inner_mean_rel_res_frac_err_fp64_ext = []

rel_time_fp32_ext = []
rel_res_frac_err_fp32_ext = []
outer_mean_rel_res_frac_err_fp32_ext = []
inner_mean_rel_res_frac_err_fp32_ext = []

for setup in SETUPS:
    for matrix in SETUP_MATRIX_MAPPING[setup]:
        for restart_param in RESTART_PARAMS:
            for exp_iter in range(N_EXPERIMENT_ITERATIONS):

                sub_data = df_sel_setup_matrix_restart(
                    extended_data, setup, matrix, restart_param
                )
                sub_data = sub_data[sub_data["experiment_iter"] == exp_iter]

                if sub_data[sub_data["solver"] == "FP FP64"].size != 0:

                    fp64_row = sub_data[sub_data["solver"] == "FP FP64"].iloc[0]

                    rel_time_fp64_ext.append(
                        sub_data["elapsed_time_ms"]/fp64_row["elapsed_time_ms"]
                    )

                    rel_res_frac_err_fp64_ext.append(
                        rel_res_frac_err(
                            sub_data["final_relres"], fp64_row["final_relres"]
                        )
                    )

                    series_mean_relres_frac_err = sub_data["outer_relres"].transform(
                            mean_rel_res_frac_err,
                            0,
                            fp64_row["outer_relres"]
                        )
                    assert series_mean_relres_frac_err[
                        sub_data["solver"] == "FP FP64"
                    ].values[0] == 0.
                    outer_mean_rel_res_frac_err_fp64_ext.append(
                        series_mean_relres_frac_err
                    )

                    series_mean_relres_frac_err = sub_data["inner_relres"].transform(
                            mean_rel_res_frac_err,
                            0,
                            fp64_row["inner_relres"]
                        )
                    assert series_mean_relres_frac_err[
                        sub_data["solver"] == "FP FP64"
                    ].values[0] == 0.
                    inner_mean_rel_res_frac_err_fp64_ext.append(
                        series_mean_relres_frac_err
                    )
                

                if sub_data[sub_data["solver"] == "FP FP32"].size != 0:

                    fp32_row = sub_data[sub_data["solver"] == "FP FP32"].iloc[0]

                    rel_time_fp32_ext.append(
                        sub_data["elapsed_time_ms"]/fp32_row["elapsed_time_ms"]
                    )

                    rel_res_frac_err_fp32_ext.append(
                        rel_res_frac_err(
                            sub_data["final_relres"], fp32_row["final_relres"]
                        )
                    )

                    series_mean_relres_frac_err = sub_data["outer_relres"].transform(
                            mean_rel_res_frac_err,
                            0,
                            fp32_row["outer_relres"]
                        )
                    assert series_mean_relres_frac_err[
                        sub_data["solver"] == "FP FP32"
                    ].values[0] == 0.
                    outer_mean_rel_res_frac_err_fp32_ext.append(
                        series_mean_relres_frac_err
                    )

                    series_mean_relres_frac_err = sub_data["inner_relres"].transform(
                            mean_rel_res_frac_err,
                            0,
                            fp32_row["inner_relres"]
                        )
                    assert series_mean_relres_frac_err[
                        sub_data["solver"] == "FP FP32"
                    ].values[0] == 0.
                    inner_mean_rel_res_frac_err_fp32_ext.append(
                        series_mean_relres_frac_err
                    )

rel_time_fp64_ext = pd.concat(rel_time_fp64_ext)
rel_time_fp64_ext.name = "rel_time_fp64"

rel_res_frac_err_fp64_ext = pd.concat(rel_res_frac_err_fp64_ext)
rel_res_frac_err_fp64_ext.name = "rel_res_frac_err_fp64"

outer_mean_rel_res_frac_err_fp64_ext = pd.concat(outer_mean_rel_res_frac_err_fp64_ext)
outer_mean_rel_res_frac_err_fp64_ext.name = "outer_mean_rel_res_frac_err_fp64"

inner_mean_rel_res_frac_err_fp64_ext = pd.concat(inner_mean_rel_res_frac_err_fp64_ext)
inner_mean_rel_res_frac_err_fp64_ext.name = "inner_mean_rel_res_frac_err_fp64"

rel_time_fp32_ext = pd.concat(rel_time_fp32_ext)
rel_time_fp32_ext.name = "rel_time_fp32"

rel_res_frac_err_fp32_ext = pd.concat(rel_res_frac_err_fp32_ext)
rel_res_frac_err_fp32_ext.name = "rel_res_frac_err_fp32"

outer_mean_rel_res_frac_err_fp32_ext = pd.concat(outer_mean_rel_res_frac_err_fp32_ext)
outer_mean_rel_res_frac_err_fp32_ext.name = "outer_mean_rel_res_frac_err_fp32"

inner_mean_rel_res_frac_err_fp32_ext = pd.concat(inner_mean_rel_res_frac_err_fp32_ext)
inner_mean_rel_res_frac_err_fp32_ext.name = "inner_mean_rel_res_frac_err_fp32"

extended_data = extended_data.join(
    (
        rel_time_fp64_ext,
        rel_res_frac_err_fp64_ext,
        outer_mean_rel_res_frac_err_fp64_ext,
        inner_mean_rel_res_frac_err_fp64_ext,
        rel_time_fp32_ext,
        rel_res_frac_err_fp32_ext,
        outer_mean_rel_res_frac_err_fp32_ext,
        inner_mean_rel_res_frac_err_fp32_ext
    )
)
display(extended_data)

extended_data.to_pickle(extended_data_path)

In [ ]:
series_mean_relres_frac_err

##### Compress into Experiment Iteration Median Values

In [ ]:
median_data = []

for setup in SETUPS:
    for matrix in SETUP_MATRIX_MAPPING[setup]:
        for restart_param in RESTART_PARAMS:

            sub_data = df_sel_setup_matrix_restart(
                extended_data, setup, matrix, restart_param
            )

            for solver in SOLVERS:

                solve_experiments = sub_data[sub_data["solver"] == solver]

                if len(solve_experiments) > 0:

                    median_experiment = {
                        "setup": setup,
                        "matrix": matrix,
                        "solver": solver,
                        "restart_param": restart_param,
                        "all_populated": bool(solve_experiments["populated"].all()),
                        "all_initiated": bool(solve_experiments["initiated"].all()),
                        "all_converged": bool(solve_experiments["converged"].all()),
                        "all_terminated": bool(solve_experiments["terminated"].all()),
                        "med_outer_iter": int(solve_experiments["outer_iters"].median()),
                        "med_inner_iter": int(solve_experiments["inner_iters"].median()),
                        "med_time_ms": int(solve_experiments["elapsed_time_ms"].median()),
                        "med_rel_res": solve_experiments["final_relres"].median(),
                        "med_rel_time_fp64": solve_experiments["rel_time_fp64"].median(),
                        "med_rel_res_frac_err_fp64": solve_experiments["rel_res_frac_err_fp64"].median(),
                        "med_outer_mean_rel_res_frac_err_fp64": (
                            solve_experiments["outer_mean_rel_res_frac_err_fp64"].median()
                        ),
                        "med_inner_mean_rel_res_frac_err_fp64": (
                            solve_experiments["inner_mean_rel_res_frac_err_fp64"].median()
                        ),
                        "med_rel_time_fp32": solve_experiments["rel_time_fp32"].median(),
                        "med_rel_res_frac_err_fp32": solve_experiments["rel_res_frac_err_fp32"].median(),
                        "med_outer_mean_rel_res_frac_err_fp32": (
                            solve_experiments["outer_mean_rel_res_frac_err_fp32"].median()
                        ),
                        "med_inner_mean_rel_res_frac_err_fp32": (
                            solve_experiments["inner_mean_rel_res_frac_err_fp32"].median()
                        )
                    }

                    median_data.append(median_experiment)

median_data = pd.DataFrame(median_data)
display(median_data)

median_data.to_pickle(median_data_path)